Vector Notebook
==============

***Un carnet rassemblant toutes mes notes sur mes travaux sur Vector***

**Vector name:** *Vector-W2B1*

**Vector serial number:** *0030263b*

**Vector OS version:** *WireOs3.0.1.32oskr*

**Vector Recovery version:** *0030263b*


# 02/02/2026
C'est la première fois que je retourne sur Vector depuis deux ans. Le robot est bloqué sur l'écran d'onboarding et quand j'essaie de le mettre à jour, il affiche un point d'exclamation dans un nuage.
## Recovery
J'avais déjà installé le recovery OpenSourceKitforRobots (OSKR) fourni par DDL : 0030263b.ota (situé dans le dossier de Backups de ce répertoire).
## DDL OS
### Récupération d'une OTA officielle de DDL compatible avec mon Vector
Je souhaite donc installé un OS officiel de DDL pour tout remettre à zéro. Il me fallait une version OSKR Escape Pod (EP).
Pour la récupérer, j'ai utilisé une image Docker de cyb3rdog : cyb3rdog/escapepod-docker:latest dans laquelle cette OTA était enregistrée. 
```bash
sudo docker run -it --rm --name escapepod -h escapepod -p 80:80 -p 8084:8084 -p 8085:8085 -p 8086:8086 -p 65533:65533 -d cyb3rdog/escapepod:latest
```
J'ai ensuite utiliser l'explorateur de fichier de Docker Desktop. L'OTA OSKR_EP_1.7.3_6016.ota était stockée dans le chemin suivant : /usr/local/escapepod/ui/ota. Je l'ai extraite et stockée dans le dossier de Backups. 
### Installation de l'OTA sur Vector
D'abord, j'ai créé un serveur web minimal dans le répertoire de mon Mac où l'OTA est stockée : 
```bash
cd /path/to/OTA/directory
python3 -m http.server
```
Puis en utilisant l'interface en command line du websetup [https://wpsetup.keriganc.com/html/main.html](https://wpsetup.keriganc.com/html/main.html) et la commande ota-start, j'ai téléchargé cette OTA sur Vector : 
```bash
wifi-scan
wifi-connect "Domicile de Rodolphe (2,4 GHz)" Domicile?Rodolphe#92130@
ota-start http://192.168.8.246:8000/OSKR_EP_1.7.3_6016.ota
```
Pour avoir la liste des fonctions disponibles, il suffit de taper la commande suivante : 
```bash
help    
```
### Onboarding 
Vector a redémarré après l'installation de l'OTA et a affiché l'écran d'onboarding. J'ai donc continué l'onboarding en utilisant l'outil officiel : [https://vector-web-setup.ddl.io/](https://vector-web-setup.ddl.io/)
## WireOS
### Présentation
WireOS est un OS alternatif pour Vector développé par Wire / kercre123. Il est basé sur le code source de la version 2.0.1.6076 de DDL qui a été leaké.
Il existe différentes méthodes d'installations : 
"
1. WireOS is available at froggitti's Dev Vector web setup under the Custom Firmware stack. 
2. Or, in the BLE terminal in recovery, after connecting him to Wi-Fi. Using https://wpsetup.keriganc.com/. 
    1. Connect to Vector following instructions
    2. Unselect Enable auto-setup flow And Enable OS auto-selection.
    3. Run command : ota-start http://ota.pvic.xyz/vic/latest/dev.ota
3. Or, while SSHed into another custome firmware (CFW):
update-os http://ota.pvic.xyz/vic/latest/dev.ota
4. Or if the robot is running OSKR:
systemctl stop anki-robot.target
mount -o rw,remount,exec /data
curl -o /data/update-engine http://ota.pvic.xyz/update-engine
chmod +rwx /data/update-engine
/data/update-engine http://ota.pvic.xyz/vic/latest/oskr.ota -v
"
### Installation de WireOS
J'ai utilisé cette dernière méthode. 
Voici la sortie que j'ai eu : 
``` Bash
Target slot is b
Manifest
Updating to version 3.0.1.32oskr
Boot
System  
```
Après le redémarrage du robot, j'ai eu une erreur 800 sur l'écran de Vector, j'ai donc redémarré le robot une nouvelle fois et WireOS est apparu sur l'écran parfaitement fonctionnel.  
J'ai ensuite téléchargé cette OTA sur mon ordinateur et enregistré dans le dossier de Backups sous le nom `20260202oskr.ota`.
### Onboarding
J'ai onboardé le robot en utilisant mon compte startus.ddl.io sur [DDL websetup](https://vector-web-setup.ddl.io/) et téléchagé les logs `vector-logs-2026-02-02-20-13-44.tar.bz2` du robot pour les conserver dans le dossier de Backups. 
### Conseils d'usage
- ddn [on/off] : Turns on/off DevDoNothing, which makes the bot stand still until shaken.
- reonboard : Puts him back into onboarding mode without fully clearing user data 
- vmesg [-c|-t] <grep args> : A wrapper for cat/tail /var/log/messages.
- temper : Simple script which tells you CPU temps
- voff : Shuts the bot off, closes your SSH session before doing so (the shutdown command just restarts the bot, this is different)
- mrw : mount -o rw,remount /
- htop
- rsync
## Paramétrage de macOS pour communiquer avec Vector
J'ai téléchargé les clefs SSH root de l'OTA OSKR_EP_1.7.3_6016.ota depuis [https://unlock-prod.froggitti.net/](https://unlock-prod.froggitti.net/) et les ai ajoutées à mon Mac. J'ai ensuite changé les permissions de la clé privée : 
```bash
chmod 400 ~/.ssh/id_rsa_Vector-W2B1
```
Puis j'ai édité le fichier ~/.ssh/config avec les lignes suivantes :
```Bash
# Vector
Host vector
        User root
        HostName 172.20.10.2
        IdentityFile ~/.ssh/id_rsa_Vector-W2B1
        HostKeyAlgorithms +ssh-rsa
        PubkeyAcceptedAlgorithms +ssh-rsa
```
Remplacer l'adresse IP du robot par vector-w2b1.local n'a pas fonctionné, j'ai donc utilisé l'adresse IP du robot que j'avais récupéré sur l'écran CCIS.   :
I had to manualy input IP adress to ~/.ssh/config, because vector-w2b1.local did not work 

# 21/02/2026
L'objectif était de développer une solution pour pouvoir faire tourner le serveur de Wire-Pod sur mon MacBook Pro, afin de pouvoir faire du développement et des tests plus facilement en utilisant soit une machine virtuelle OrbStack, soit un container Docker. 

## Création d'un fork du projet wire-pod
J'ai commencé par créer un fork du projet wire-pod de Kercre123 sur GitHub, afin de pouvoir faire des modifications et des tests sans impacter le projet original.
Au sein de ce projet, j'ai créé une branche personnelle appelée "personal" pour y faire mes développements et mes tests sans modifier le projet original de Kercre123 pour pouvoir faire des pull requests au projet original plus tard si nécessaire et bénéficier des mises à jour de Kercre123.

## Première tentative en utilisant une machine virtuelle OrbStack
J'ai créé une machine virtuelle OrbStack basée sur Ubuntu 22.04 pour faire tourner le serveur de Wire-Pod. J'ai suivi les étapes suivantes :
1. J'ai installé OrbStack sur mon MacBook Pro.
2. J'ai créé une machine virtuelle appelée EscapePod basée sur Ubuntu 22.04.
3. J'ai installé serveur Wire-Pod en me basant sur la documentation de Kercre123 : 
```bash
sudo apt install lld wget
wget https://github.com/kercre123/WirePod/releases/download/v1.2.15/wirepod_amd64-v1.2.15.deb
sudo apt install -y ./wirepod_amd64-v1.2.15.deb
```
Toute l'installation s'est déroulée sans problème et j'ai pu lancer le serveur wire-pod au lien https://escapepod.orb.local:8080.
4. J'ai configuré le serveur en utilisant escapepod.local comme adresse IP. 
5. Le set-up du robot utilisant SSH s'est déroulé simplement.
6. Au moment d'authentifier le robot, ce dernier ne parvient pas à communiquer avec le serveur. En effet, il n'existe pas de mode bridged pour les machines virtuelles OrbStack, ce qui empêche le robot d'accéder au serveur en utilisant escapepod.local comme adresse IP.
7. Afin de contourner ce problème, j'ai modifier le fichier host de Vector en me connectant en SSH au robot.
```bash
mrw
echo "192.168.8.233 escapepod.local" >> /etc/hosts
```
192.168.8.233 est l'adresse IP de mon MacBook Pro sur lequelle OrbStack est installé. Elle est définie comme "fixe avec DHCP" dans les paramètres WiFi de macOS. Attention, cela ne fonctionne que quand je suis conencté au réseau WiFi de mon domicile "Domicile de Rodolphe (5 GHz)", si je me connecte à un autre réseau, l'adresse IP de mon MacBook Pro doit être changée sinon je n'ai plus accès à internet et il faut mettre à jour le fichier host du robot en conséquence.
6. J'arrive à faire communiquer le robot avec le serveur wire-pod en utilisant escapepod.local comme adresse IP et l'authentification du robot fonctionne. Néanmoins, il est impossible d'accèder au "Settings" du robot depuis l'interface web du serveur wire-pod car l'adresse IP du robot n'est pas enregistrée dans le serveur Wire-Pod sans que je ne parvienne à comprendre pourquoi. Elle est remplacée par "[" dans les logs. 
7. Il n'est pas possible sur macOS de créer plusieurs alias .local. J'ai donc essayé de faire un container Docker (mDNS-alias-publisher-container) avec Avahi qui permettait de faire du mDNS en associant l'adresse IP du MacBook avec escapepod.local et ainsi faire en sorte que le robot puisse accéder au serveur wire-pod en utilisant escapepod.local comme adresse IP, mais cela n'a pas fonctionné non plus.
## Seconde tentative en utilisant une machine virtuelle UTM
UTM permet un accès réseau de la machine virtuelle en mode "bridged", ce qui permet au robot d'accéder au serveur wire-pod comme si la machine virtuelle était un ordinateur du réseau. 
J'ai suivi les mêmes étapes que précédemment : 
1. J'ai créé une machine virtuelle appelée EscapePod basée sur Ubuntu 22.04.
2. J'ai installé serveur Wire-Pod en me basant sur la documentation de Kercre123 : 
```bash
sudo apt install lld wget
wget https://github.com/kercre123/WirePod/releases/download/v1.2.15/wirepod_amd64-v1.2.15.deb
sudo apt install -y ./wirepod_amd64-v1.2.15.deb
```
Toute l'installation s'est déroulée sans problème et j'ai pu lancer le serveur wire-pod au lien https://escapepod.orb.local:8080.
3. J'ai configuré le serveur en utilisant le mode utilisant escapepod.local comme adresse IP.
4. Le set-up du robot utilisant SSH s'est déroulé simplement.
5. Réalisation de l'authentification du robot, qui fonctionne parfaitement et permet d'accéder au "Settings" du robot depuis l'interface web du serveur wire-pod.
Néanmoins, la machine virtuelle UTM est très lente et peu réactive, ce qui rend le développement et les tests très difficiles. De plus, la machine virtuelle UTM consomme beaucoup de ressources, ce qui impacte les performances de mon MacBook Pro.
## Troisième tentative en utilisant un container Docker OrbStack
De retour sur OrbStack, j'ai décidé d'essayer de faire tourner le serveur Wire-Pod dans un container Docker, afin de bénéficier des avantages de Docker tout en utilisant OrbStack comme environnement de développement.
1. Tout d'abord j'ai modifié le docker compose.yaml du projet wire-pod de Kercre123 dans mon répertoire forké qui présentait une erreur de déclaration des volumes en rajoutant les lignes : 
```yaml
  wire-pod-images:
    driver: local
```
J'ai fait une demande de pull request à Kercre123 pour que cette modification soit intégrée dans le projet original.
2. Ensuite, j'ai suivi les étapes suivantes :
```bash
cd wire-pod
docker compose up -d
```
3. Toute l'installation s'est déroulée sans problème mais j'ai rencontré les mêmes erreurs que lors de ma première tentative avec la machine virtuelle OrbStack, à savoir que le robot ne parvient pas à communiquer avec le serveur Wire-Pod en utilisant escapepod.local comme adresse IP.
4. J'ai en suite fait tourner le serveur Wire-Pod dans un container Docker en utilisant le mode "host" pour le réseau, ce qui permet au container Docker d'utiliser le réseau de l'hôte (mon MacBook Pro) et ainsi permettre au robot d'accéder au serveur Wire-Pod en utilisant escapepod.local comme adresse IP. Cela n'a pas non plus fonctionné. 
5. J'ai donc eu l'idée de créer une machine virtuelle UTM pour faire tourner le serveur Wire-Pod dans un container Docker, afin de pouvoir extraire le volume wire-pod-data après avoir authentifié le robot et les utiliser dans un container Docker sur OrbStack.J'ai précédemment essayé de réaliser ces étapes sur mon Raspberry Pi mais l'architecture du processeur n'étant pas la mêmen je n'ai pas réussi à avoir un container Docker fonctionnel. Après avoir créer un container de wire-pod, supprimé les modifications du fichier hosts du robot puis connecté et authentifié le robot j'ai extrait le volume wire-pod-data de la machine virtuelle UTM en utilisant les commandes suivantes : 
```bash
docker run --rm -v wire-pod-data:/data -v $(pwd):/backup alpine tar czf /backup/wire-pod-data-volume.tar.gz -C /data .
scp wire-pod-data-volume.tar.gz rodolphe@macbook-pro-de-rodolphe:/Users/rodolphe/Desktop
```
6. J'ai ensuite copier l'ensemble des fichiers du volume wire-pod-data dans le volume wire-pod-data du container Docker sur OrbStack, de nouveau modifié le fichier hosts du robot puis redémarré le container. 
C'est cette dernière méthode qui a fonctionné. Le robot est maintenant connecté à wire-pod qui tourne dans un container Docker sur OrbStack, et j'ai accès à tous les paramètres du robot depuis l'interface web du serveur Wire-Pod. 
L'adresse IP du robot est 192.168.8.244 et celle du MacBook Pro sur lequel OrbStack est installé est
192.168.8.233. 


# 22/02/2026
## Connection du robot à un autre réseau WiFi
Je souhaite connecter mon robot à un autre réseau WiFi que celui de mon domicile et notamment au partage de connection de mon iPhone. 
1. Tout d'abord j'ai connecté mon MacBook Pro au réseau WiFi de mon iPhone. L'adresse IP de mon MacBook Pro est désormais 172.20.10.3. 
2. Pour connecter le robot au même réseau WiFi j'ai utilisé sur Chrome, l'interface en command line du websetup `https://wpsetup.keriganc.com/html/main.html` et la commande wifi-connect. J'ai récupérer l'adresse IP du robot sur l'écran CCIS. L'adresse IP du robot est désormais 172.20.10.2. 
3. J'ai ensuite modifié le fichier hosts du robot pour faire en sorte que le robot puisse accéder au serveur Wire-Pod en utilisant escapepod.local comme adresse IP. 
4. Dans les volumes du container Docker wire-pod sur OrbStack, j'ai modifié l'addresse IP du robot dans le fichier wire-pod-data/chipper/jdocs/botSdkInfo.json.
5. J'ai également eu besoin d'éditer mon fichier .ssh/config sur macOS.
## Création du répertoire Vector
L'objectif de ce répertoire est de contenir tous les répertoires et les fichiers liés à mon travail sur Vector. 
### Liaison du répertoire wire-pod à l'aise de git submodule
Pour intégrer le répertoire de wire-pod dans mon projet Vector, j'ai utilisé la commande suivante :
```bash
pwd && git rev-parse --is-inside-work-tree && git submodule add -b personal https://github.com/rodolphemds/wire-pod.git wire-pod && git status --short && echo '--- .gitmodules ---' && cat .gitmodules
git commit -m "Add wire-pod submodule on personal branch"
git push
```
Cette commande fait plusieurs choses :
1. Affiche le chemin actuel et vérifie que je suis dans un dépôt git.
2. Ajoute le répertoire wire-pod en tant que sous-module à partir de la branche "personal" du dépôt GitHub.
3. Affiche le statut du dépôt pour vérifier que le sous-module a été ajouté correctement.
4. Affiche le contenu du fichier .gitmodules pour confirmer que le sous-module est bien référencé.
5. Commits les changements avec un message descriptif.
6. Pousse les changements vers le dépôt distant.    
Pour cloner ce répertoire avec les submodules, il faudra utiliser la commande suivante :
```bash
git clone --recurse-submodules <repository-url>
```
### Liaison du répertoire wirepod-vector-python-sdk
J'ai d'abord créé un fork du répertoire kercre123/wirepod-vector-python-sdk dans mon GitHub puis l'ai ajouté en tant que sous-module dans le répertoire Vector en utilisant la même méthode que pour le répertoire wire-pod:
```bash
git submodule add https://github.com/rodolphemds/vector-wirepod-python-sdk.git vector-wirepod-python-sdk && git status --short && echo '--- .gitmodules ---' && cat .gitmodules
git commit -m "Add vector-wirepod-python-sdk submodule"
git push
git submodule update --init --recursive vector-wirepod-python-sdk
```
### Liaison du répertoire wirepod-vector-python-sdk
J'ai d'abord créé un fork du répertoire claudix29/available-anki-vector-documentation dans mon GitHub puis l'ai ajouté en tant que sous-module dans le répertoire Vector :
```bash
git submodule add https://github.com/rodolphemds/available-anki-vector-documentation.git available-anki-vector-documentation && git status --short && echo '--- .gitmodules ---' && cat .gitmodules
git commit -m "Add available-anki-vector-documentation submodule"
git push
git submodule update --init --recursive available-anki-vector-documentation
```
### Liaison du répertoire vectorOS
J'ai d'abord créé un fork du répertoire os-vector/wire-os dans mon GitHub. J'ai modifié les liens des submodules dans .gitmodules pour qu'ils redirigent vers mes répertoires GitHub. J'ai eu besoin de forké les répertoires suivants : os-vector/wired, purpl-org/purplpkg, kercre123/vic-cloudless et os-vector/wire-os-victor. Au sein du fichier .gitmodules du répertoire rodolphemds/vectorOS-anki, j'ai modifié l'adresse du submodules EXTERNALS pour qu'il pointe vers rodolphemds/vectorOS-externals au lieu de os-vector/wire-os-externals.
J'ai en suite ajouté le répertoire rodolphemds/vectorOS en tant que sous-module dans le répertoire Vector :
```bash
git submodule add https://github.com/rodolphemds/vectorOS.git vectorOS && git status --short && echo '--- .gitmodules ---' && cat .gitmodules
git commit -m "Add vectorOS submodule"
git push
git submodule update --init --recursive vectorOS
```
### Liaison du répertoire vectorOS-externals
J'ai d'abord créé un fork du répertoire os-vector/wire-os-externals dans mon GitHub puis l'ai ajouté en tant que sous-module dans le répertoire Vector :
```bash
git submodule add https://github.com/rodolphemds/vectorOS-externals.git vectorOS-externals && git status --short && echo '--- .gitmodules ---' && cat .gitmodules
git commit -m "Add vectorOS-externals submodule"
git push
git submodule update --init --recursive vectorOS-externals
```
## Création d'un Jupyter Notebook 
J'ai créé un Jupyter Notebook pour documenter mes travaux sur Vector. Ce notebook me permettra de suivre mes progrès, de noter les idées et les problèmes rencontrés, et de partager mes résultats.
Pour créer un environnement virtuel Python dans VSCode qui me servira a faire fonctionner le Notebook Jupyter et le code du SDK, j'ai suivi les étapes suivantes :
1. Installation de la dernière version de Python 3 sur macOS à partir de homebrew et de la version de Python 3.12.0 pour être compatible avec le SDK de Vector :
```bash
brew install python
brew install python@3.12
``` 
2. Création d d'un nouveau Notebook Jupyter dans le répertoire Vector.
3. Sélection d'un nouveau kernel Python pour le Notebook.
4. Installation de l'extension Python pour VSCode si ce n'est pas déjà fait.
5. A partir de la palette de commandes (Ctrl+Shift+P), j'ai sélectionné "Python: Create Environment" et choisi l'option ".venv".
6. VSCode a automatiquement créé un environnement virtuel dans le répertoire Vector et l'a activé pour le Notebook. L'environnement est basé sur Python 3.12.12. 
## Installation du Python SDK de Vector fonctionnant avec WirePod
### Installation du SDK
Dans mon environnement virtuel, j'ai installé le Python SDK de Vector en utilisant :
```bash
pip install wirepod_vector_sdk
pip install "wirepod_vector_sdk[3dviewer]"
```
Pour mettre à jour le SDK, il faudra utiliser la commande suivante :
```bash 
pip install wirepod_vector_sdk --upgrade
```
### Pour authentifier le robot avec le SDK, j'ai utilisé la commande suivante :
```bash    
python -m anki_vector.configure
```
J'ai obtenu l'erreur suivante : 
``
Downloading Vector certificate from wire-pod... DONE
Writing certificate file to '/Users/rodolphe/.anki_vector/Vector-W2B1-0030263b.cert'...

Attempting to download guid from Vector-W2B1 at 172.20.10.2:443... ERROR

Failed to authorize request:
Please be sure to first set up Vector using the companion app.
``
Le GUID du robot exrait de botSdkInfo.json est "/CCGr3+bq5xHebeNW9oOTQ==". Je vais le passer directement dans le fichier de configuration du SDK pour voir si cela résout le problème d'authentification. 
J'ai donc modifié le fichier configure.__main__.py du SDK pour y ajouter le nom, le GUID, l'addresse IP du robot et celle de wire-pod.
Puis j'ai lancé les commandes suivantes pour réinstaller le package du SDK en mode éditable et relier l'environnement de développement au dossier du projet pour qu'au lieu de copier le code du SDK dans .venv/site-packages, pip crée un raccourci qui pointe vers mon dossier local. Si je modifie un fichier du SDK, ainsi les changements sont immédiatement pris en compte sans réinstallation :
```bash
python -m pip install -e ./vector-wirepod-python-sdk
python -m anki_vector.configure
```
L'authentification ne rapporte pas d'erreur. J'ai pu testé les exemples de code du SDK et ils fonctionnent parfaitement.
### Génération de la documentation
J'ai d'abord installé Sphinx pour générer la documentation du SDK :
```bash
cd vector-wirepod-python-sdk/docs/
pip install -r requirements.txt
```
Puis j'ai généré la documentation en utilisant la commande suivante :
```bash
make clean
make html
```
Cela n'a pas fonctionné compte tenu de la version de Sphinx utilisée, j'ai donc modifié la version de Sphinx dans le fichier requirements.txt pour utiliser une version compatible avec la version de Python 3.12.12 que j'utilise dans mon environnement virtuel, puis j'ai relancé la commande make html qui a fonctionné correctement.
La documentation est maintenant disponible dans le répertoire `vector-wirepod-python-sdk/docs/.build/index.html`. 
## Exploration des différents serveurs présents sur le robot
Il existe plusieurs serveurs qui tournent sur le robot Vector comme le décrit /Docs/development/web-server.md au sein du répertoire [vector-os-anki.git](https://github.com/rodolphemds/vectorOS-anki.git), notamment :
- Port 8888: Le serveur web qui s'exécute dans le processus engine, et qui permet d'accéder aux variables et fonctions de la console du processus engine.
- Port 8889: Le serveur web qui s'exécute dans le processus anim, et qui permet d'accéder aux variables et fonctions de la console du processus anim.
- Port 8080: Wired pour configurer des paramètres additionnels du robot. 
Il est possible de les modifier grâce à la fonction :
```bash
curl "http://robot_ip:8888/consolevarset?key=MirrorMode&value=1"
```
Cette commande peut être utilisée directement sur le robot.
## Modification des lumières du backpack de Vector
Exécuter sur le robot
```Bash
touch /data/data/enableankilights
reboot
```
## Version locale de websetup
### Liaison du répertoire vector-websetup
J'ai d'abord créé un fork du répertoire froggitti/websetup.froggitti.net dans mon GitHub puis l'ai ajouté en tant que sous-module dans le répertoire Vector :
```bash
git submodule add https://github.com/rodolphemds/vector-websetup.git vector-websetup && git status --short && echo '--- .gitmodules ---' && cat .gitmodules
git commit -m "Add vector-websetup submodule"
git push
git submodule update --init --recursive vector-websetup
```
L'idée est de créer un container Docker sur Orbstack permettant de faire fonctionner une copie locale de websetup. 
### Choix de l'image Docker de nodejs
Je choisis node:22-slim basé sur Débian Bookworm-slim, meilleur compromis entre légèreté et compatibilité avec websetup ([A Deeper Look into Node.js Docker Images: Help, My Node Image Has Python!](https://labs.iximiuz.com/tutorials/how-to-choose-nodejs-container-image#conclusion)). 
### Création du Dockerfile
Voici le contenu du Dockerfile que j'ai créé pour faire fonctionner websetup dans un container Docker :
```Dockerfile
FROM node:22-slim
WORKDIR /app
RUN npm install vector-web-setup
RUN /app/node_modules/.bin/vector-web-setup configure
ENV PORT=7000
EXPOSE ${PORT}
CMD ["sh", "-c", "/app/node_modules/.bin/vector-web-setup serve -p ${PORT} "]
```
J'ai choisi le port 7000 pour éviter les conflits avec d'autres services wire-pod qui utilisent le port 80000.
### Modification des lignes suivantes comme expliqué dans le README.md :
- Remplacer [line 160 in /rts-js/main.js] par `http://vector-websetup.vector-websetup.orb.local:7000/static/firmware/${_stack.name}/`
- Remplacer [line 1017 in /site/js/rts.js] par `http://vector-websetup.vector-websetup.orb.local:7000/static/firmware/${_stack.name}/`
- Remplacer [line 1077 in /site/js/rts.js] par `http://vector-websetup.vector-websetup.orb.local/firmware`
### Construction et lancement du container Docker
J'ai ensuite créeer un fichier docker-compose.yml puis construit l'image Docker à partir du Dockerfile et lancé le container Docker en utilisant les commandes suivantes :
```bash
docker compose up -d
```
### Accès à websetup
En utilisant Chrome, j'ai accédé à websetup en utilisant l'adresse suivante : [http://vector-websetup.vector-websetup.orb.local:7000/](http://vector-websetup.vector-websetup.orb.local:7000/).
Le site ne détectait pas la connexion bluetooth compte tenu des restrictions de sécurité de Chrome sur les sites non sécurisés, j'ai donc utilisé l'adresse [https://vector-websetup.vector-websetup.orb.local:7000/](https://vector-websetup.vector-websetup.orb.local:7000/) avec succès. 
En désactivant la case auto-setup flow, je peux accéder à l'interface en command line
### Concernant le container
Sur le terminal du container Docker, je peux accéder à la commande /app/node_modules/.bin/vector-web-setup help qui liste toutes les possibilités de la commande vector-web-setup pour paramétrer le serveur. 
## Compilation d’un premier script en C 
Le but est d’écrire un premier script en C et de le compiler pour l’exécuter sur Vector. 
J’ai créé une nouvelle machine virtuelle sur OrbStack appelée Ubuntu basée sur Ubuntu Noble puis, conformément aux instructions sur [https://modder.my.to/oskr-owners-manual/doc/cross-compilation.html](https://modder.my.to/oskr-owners-manual/doc/cross-compilation.html), j’ai téléchargé sur la machine virtuelle le fichier gcc-linaro-6.3.1-2017.05-x86_64_arm-linux-gnueabi.tar.xz que j’ai enregistré dans mes archives. Je l’ai ensuite extrait et installé selon les instructions suivantes : 
```Bash
sudo apt install wget xz-utils
wget  https://releases.linaro.org/components/toolchain/binaries/6.3-2017.05/arm-linux-gnueabi/gcc-linaro-6.3.1-2017.05-x86_64_arm-linux-gnueabi.tar.xz
```
Je l’extraie et l’inclus à mon PATH :
```Bash
tar xJf gcc-linaro-6.3.1-2017.05-x86_64_arm-linux-gnueabi.tar.xz
export PATH="${PATH}:/home/rodolphe/gcc-linaro-6.3.1-2017.05-x86_64_arm-linux-gnueabi/bin"
```
Il faudra que j’exécute cette ligne à chaque fois que je compile pour Vector 

Je crée un simple hello world program : 
```Bash
cat > hello.c << _DONE_
#include <stdio.h>
void main() {
 printf("hello, vector!\r\n");
}
_DONE_
```
Je le compile : 
```Bash
arm-linux-gnueabi-gcc -o hello hello.c
```
Et je le copie ensuite en utilisant l’explorateur de fichiers de OrbStack dans un dossier de mon MacBook Pro avant de le transférer sur Vector en utilisant la fonction :
```Bash
scp  hello root@vector:/data/
```
Pour l’exécuter sur le robot, je rend la partition /data executable :
```Bash
mount -o remount,exec /data
```
Et je l’exécute :
```Bash
/data/hello
hello, vector!
```

## Préparation de l'espace de travail pour compiler le code de VectorOS
Cela me permettra de créer et de compiler mon propre OS pour Vector.
J’installe Docker et git sur ma machine Ubuntu virtuelle OrbStack :
```Bash
sudo apt install docker.io docker-compose-v2 git
```
Je m’assure que Docker puisse être utilisé par l’utilisateur courant :
```Bash
sudo groupadd docker
sudo gpasswd -a $USER docker
newgrp docker
sudo chown root:docker /var/run/docker.sock
sudo chmod 660 /var/run/docker.sock
```
Cela me permettra, conformément aux instructions de Kercre123 de compiler l’OS de Vector. 

# 23/02/2026
## Nettoyage des submodules 
Pour des soucis de gain de place sur mon disque dur, j'ai supprimé les sous-modules vectorOS-anki et vectorOS-externals qui étaient compris à l'intérieur du submodule vectorOS. 
```bash
# On enlève d’abord les sous-modules de .git/config avec :
git submodule deinit vectorOS-anki
git submodule deinit vectorOS-externals
# puis on supprime le répertoire du sous-module du répertoire .git/modules :
rm -rf .git/modules/vectorOS-anki
rm -rf .git/modules/vectorOS-externals
# enfin on supprime répertoire ainsi que l’entrée dans le .gitmodules avec :
git rm vectorOS-anki
git rm vectorOS-externals
# j'actualise ensuite tous les submodules :
git submodule update --init --recursive --remote
```

# Prochaines étapes : 
Regarder : 
- https://github.com/rodolphemds/vector-go-sdk.git
- https://github.com/kercre123/the-best-vector-control-program.git
- https://github.com/rodolphemds/vector-go-sdk.git
- https://github.com/rodolphemds/vector-go.git
- https://github.com/rodolphemds/vector-gobot.git
